<a href="https://colab.research.google.com/github/Jameslorry6666/Micheal.iml/blob/master/Numba_Data_Minning_cuda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#High-performance data processing using the Namba library


# Installing the library
!pip install numba

In [2]:
!pip install numba

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
from numba import cuda
from timeit import default_timer as timer
print(cuda.gpus)



<Managed Device 0>


In [20]:

from numba import cuda
gpu = cuda.get_current_device()
print("name = %s" % gpu.name)
print("maxThreadsPerBlock = %s" % str(gpu.MAX_THREADS_PER_BLOCK))
print("maxBlockDimX = %s" % str(gpu.MAX_BLOCK_DIM_X))
print("maxBlockDimY = %s" % str(gpu.MAX_BLOCK_DIM_Y))
print("maxBlockDimZ = %s" % str(gpu.MAX_BLOCK_DIM_Z))
print("maxGridDimX = %s" % str(gpu.MAX_GRID_DIM_X))
print("maxGridDimY = %s" % str(gpu.MAX_GRID_DIM_Y))
print("maxGridDimZ = %s" % str(gpu.MAX_GRID_DIM_Z))
print("maxSharedMemoryPerBlock = %s" % str(gpu.MAX_SHARED_MEMORY_PER_BLOCK))
print("asyncEngineCount = %s" % str(gpu.ASYNC_ENGINE_COUNT))
print("canMapHostMemory = %s" % str(gpu.CAN_MAP_HOST_MEMORY))
print("multiProcessorCount = %s" % str(gpu.MULTIPROCESSOR_COUNT))
print("warpSize = %s" % str(gpu.WARP_SIZE))
print("unifiedAddressing = %s" % str(gpu.UNIFIED_ADDRESSING))
print("pciBusID = %s" % str(gpu.PCI_BUS_ID))
print("pciDeviceID = %s" % str(gpu.PCI_DEVICE_ID))



name = b'Tesla T4'
maxThreadsPerBlock = 1024
maxBlockDimX = 1024
maxBlockDimY = 1024
maxBlockDimZ = 64
maxGridDimX = 2147483647
maxGridDimY = 65535
maxGridDimZ = 65535
maxSharedMemoryPerBlock = 49152
asyncEngineCount = 3
canMapHostMemory = 1
multiProcessorCount = 40
warpSize = 32
unifiedAddressing = 1
pciBusID = 0
pciDeviceID = 4


In [21]:
from numba import cuda
@cuda.jit
def dec_kernel(x, y, out):
    
    idx = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x 
    out[idx] = x[idx] - y[idx]



In [22]:
import numpy as np
import math

n = 1000000 
x = np.arange(n).astype(np.int32) # vectort х [0...1000000] 
y = np.ones_like(x)               # [1...1]

d_x = cuda.to_device(x) # copy to data of the video card
d_y = cuda.to_device(y) # copy to data of the video card
d_out = cuda.device_array_like(d_x) # the resulting vector

 # setting the dimension of the grid
threads_per_block = 256
blocks_per_grid = math.ceil(n / threads_per_block)

In [24]:
start = timer()
dec_kernel[blocks_per_grid, threads_per_block](d_x, d_y, d_out) # starting the kernel
print("GPU:", timer()-start)

GPU: 0.00040332300000045507


In [25]:
print(d_out.copy_to_host())

[    -1      0      1 ... 999996 999997 999998]


In [26]:
out  = np.ones(n, dtype = np.int32)

def dec_cpu(x, y, out):
    for i in range(n):
        out[i] = x[i]-y[i]

In [27]:
start = timer()
dec_cpu(x, y, out)
print("CPU:", timer()-start)

CPU: 0.2778614159999506
